In [1]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import time
import numpy as np
import os

In [2]:
feature_compare = pd.DataFrame()
feat_indices = [i for i in range(11)]


In [3]:
# dta = pd.read_csv("training_variables/S_Material_Class.csv", index_col=0)
# dta = pd.read_csv("training_variables/S_c_m_Net_Value_A.csv", index_col=0)
# dta = pd.read_csv("training_variables/SOP_j_c_m_Net_Value_A.csv", index_col=0)
# dta = pd.read_csv("training_variables/SOP_j_Material_Class.csv", index_col=0)
dta = pd.read_csv("training_variables/SOP_j_Material_Classc1.csv", index_col=0)

sorted_dta = pd.DataFrame()
classification = pd.read_csv("classification.csv")
for id in classification.index:
    cus_id = classification.loc[id, "Customer_ID"]
    new_row = dta.loc[dta["Customer_ID"]==cus_id]
    sorted_dta = pd.concat([sorted_dta, new_row])
    
dta = sorted_dta
test_all = dta[dta["Reseller"].isna()]
dta = dta[~dta["Reseller"].isna()]
y = dta["Reseller"]
# x = np.append(x, dta.drop(columns=["Customer_ID", "Reseller", "Test_set_id"]).to_numpy(), axis=1)
x = dta.drop(columns=["Customer_ID", "Reseller", "Test_set_id"]).to_numpy()
# x = dta.drop(columns=["Customer_ID",  "V1", "Reseller.x", "Reseller.y",  "Test_set_id.x", "Test_set_id.y"]).to_numpy()

In [4]:
x

array([[      0.        ,  493850.33333333,       0.        , ...,
              0.        ,       0.        ,       0.        ],
       [ 997242.        ,       0.        ,  130113.52380952, ...,
              0.        ,       0.        ,       0.        ],
       [      0.        ,       0.        ,       0.        , ...,
              0.        ,       0.        ,       0.        ],
       ...,
       [      0.        , 1177246.        ,  322435.5       , ...,
              0.        ,       0.        ,       0.        ],
       [      0.        ,       0.        ,       0.        , ...,
              0.        ,       0.        ,       0.        ],
       [      0.        ,       0.        ,       0.        , ...,
              0.        ,       0.        ,       0.        ]])

In [5]:
x.shape

(2049, 328)

In [6]:
# for feat in feat_indices:
# y = dta[:, 329, 0]
# x = dta[:, 0:329, feat].reshape(len(dta), -1)

# 划分训练集和测试集
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

# 决策树分类器
dec_tree = DecisionTreeClassifier()
dec_tree.fit(x_train, y_train)

print("*" * 30 + " 准确率 " + "*" * 30)
print(dec_tree.score(x_test, y_test))





****************************** 准确率 ******************************
0.8585365853658536


In [7]:
def CaculateAcc(reselt, label):
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    for i in range(len(reselt)):
        if(label[i]==1):
            if(reselt[i]==1):
                TP = TP+1
            else: FN =FN+1
        else:
            if(reselt[i]==1):
                FP = FP+1
            else: TN =TN+1
    TPR = TP/(TP+FN)
    TNR = TN/(FP+TN)
    BAC = (TPR+TNR)/2
    return BAC

In [8]:
result = dec_tree.predict(x_test)
label = np.array(y_test)
acc = CaculateAcc(result, label)
acc

0.5710041592394534

In [ ]:
# 随机森林分类器
# n_jobs: -1表示设置为核心数量
rf = RandomForestClassifier(n_jobs=-1)

# 网格搜索
# n_estimators: 决策树数目
# max_depth: 树最大深度
param = {
    "n_estimators": [120, 200, 300, 500, 800, 1200],
    # "max_depth": [5, 8, 15, 25, 30]
    "max_depth": [5, 10, 11, 12, 15, 20]
}

# 2折交叉验证
search = GridSearchCV(rf, param_grid=param, cv=2)
print("*" * 30 + " 超参数网格搜索 " + "*" * 30)

start_time = time.time()
search.fit(x_train, y_train)

print("耗时：{}".format(time.time() - start_time))
print("最优参数：{}".format(search.best_params_))

# print("*" * 30 + " 准确率 " + "*" * 30)
# print(search.score(x_test, y_test))
# feature_compare.loc[feat, "dt_acc"] = dec_tree.score(x_test, y_test)
# feature_compare.loc[feat, "rf_grid_search"] = search.score(x_test, y_test)



In [ ]:
result = search.predict(x_test)
label = np.array(y_test)
acc = CaculateAcc(result, label)
acc

In [8]:
feature_compare

""


In [20]:
x_test_f

,Customer_ID,NA.x,sales_organization_A.x,sales_organization_B.x,Order_Num.x,Variance.x.x,Total_netvalue.x.x,Variance.y.x,Total_netvalue.y.x,V1.x.x,...,336,339,341.y,342,343,344,345,346,347.y,Test_set_id
1596,0x9A208CC2D20277611D1E8EE55FDB5A11CBEC69E206C1...,0,12,0,12,1.300000e+12,22405554,0.00,0,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,11.0
64,0x0551930C36A0EC57869785D5175F371B9A99C3678BD6...,0,0,5,5,1.100000e+10,4670711,0.00,5,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,14.0
2514,0xFA69B4C9439A29094FFDDF578B6A2B76C97E8A2E5F0E...,0,33,26,59,4.700000e+11,64695037,7.00,112,-1,...,-1,-1,-1,-1,2,-1,-1,-1,-1,17.0
1346,0x808CBED146D64C12BA401959F6ED855A344CAF562BF7...,0,0,6,6,1.400000e+10,5591829,0.97,13,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,24.0
964,0x5BF5733DBE6C90E285AE10B4A6E47F0B8F2A7537F02B...,0,4,0,4,5.800000e+10,2589788,0.00,4,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
732,0x46492E2D470D1E3CB3D86A97AD694A4957BA1FF98D80...,0,13,25,38,8.900000e+11,55334309,1.70,63,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,2533.0
1759,0xAAA08E0CFB1458CF76D31F73F68A65DBA9B3CA1A8A96...,0,590,0,590,4.700000e+11,653765758,93.00,2070,-1,...,-1,-1,-1,-1,4,-1,-1,-1,-1,2535.0
2258,0xDD0A72152FBE33EFC9849363B871744067BA0C24B8C7...,0,0,424,424,2.800000e+11,555942274,30.00,1164,-1,...,-1,-1,-1,-1,-1,-1,-1,-1,-1,2538.0
1319,0x7E18FAFCE58C193B296C20540562DF0107ECAAB96351...,0,28,194,222,3.500000e+11,238442694,2.30,366,-1,...,-1,-1,-1,-1,2,-1,-1,-1,-1,2551.0


In [22]:

y_test_f = test_all['Reseller']
labels = list(test_all.columns.values)
labels.remove("Reseller")
x_test_f = test_all[labels]
x_test_f = x_test_f.drop(columns=["Customer_ID", "Test_set_id"]).to_numpy()
result = dec_tree.predict(x_test_f)




In [79]:
result
np.shape(result)
result_f = result[:]
result_f = result_f.astype(int)
np.shape(result_f)
result_f

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,

In [ ]:
path = "result"
if not os.path.exists(path):
    os.mkdir(path)

In [80]:
df = classification.loc[~np.isnan(classification["Test_set_id"]), ["Test_set_id"]]
# df.to_csv("submission_2nd.csv")
df = df.convert_dtypes(int)
df = df.reset_index(drop=True, names=["Test_set_id"])
df.insert(1, column="prediction", value=result_f)
df.to_csv("result/submission.csv", sep=";", index=False)